In [1]:
import pandas as pd
import numpy as np

## 1. Create mapping for all systems

In order to obtain the original spellings of the systems we will use the following mapping:

In [2]:
df_systems = pd.read_csv('../data/source_datasets/umbenennung-systeme.csv', encoding='utf-8', sep=';')
df_systems = df_systems.sort_values(by='name-in-tab', key=lambda x: x.str.lower())
df_systems.reset_index(drop=True, inplace=True)

In [3]:
systems_mapping = df_systems.set_index(df_systems.columns[0]).squeeze().to_dict()

## 2. Load data

In [4]:
path_source = '../data/source_datasets'
path_target = '../data/target_datasets'

In [5]:
df_main = pd.read_csv(f'{path_target}/east_middle_europe_magazines.csv', encoding='utf-16')
df_csdb = pd.read_csv(f'{path_source}/diskmags_csdb.csv', encoding='utf-8')
df_demozoo = pd.read_csv(f'{path_source}/diskmags_demozoo.csv', encoding='utf-8')
df_internet_archive = pd.read_csv(f'{path_source}/diskmags_internet_archive.csv', encoding='utf-8')
df_pouet = pd.read_csv(f'{path_source}/diskmags_pouet.csv', encoding='utf-8')
df_zxpress = pd.read_csv(f'{path_source}/diskmags_zxpress.csv', encoding='utf-8')

## Collect metadata about each issue from all datasets and merge them to one comprehensive dataset

In [6]:
column_order = ['title_cleaned', 'title_orig', 'magazine', 'release_date', 'source', 'language', 'origin', 'system_detailed', 'group', 'link']

In [7]:
df_issues_all = pd.DataFrame(columns=column_order)

In [8]:
for index, row in df_main.iterrows():
    title = row['Title']
    language = row['Magazine[Language]']
    system = row['Magazine[Systems]']
    source = row['Magazine[Source]'].split('; ')
    origin = row['Magazine[Origin]']
    
    if 'CSDB' in source:
        subset_csdb = df_csdb[df_csdb['title']==title].copy()
        subset_csdb['language'] = language
        subset_csdb['origin'] = origin
        subset_csdb['system_detailed'] = 'Commodore 64'
        subset_csdb['title_cleaned'] = subset_csdb['issue'].apply(lambda x: f'{x if not "#" in x or x.startswith("#") else x.replace("#", "No.")}')
        subset_csdb['source'] = 'CSDB'
        subset_csdb['group'] = subset_csdb['group'].apply(lambda x: '; '.join(x.split(', ')) if isinstance(x, str) else x)
        subset_csdb = subset_csdb.rename(columns={'issue': 'title_orig',
                                                  'title': 'magazine',
                                                  'release_converted': 'release_date'})
        subset_csdb = subset_csdb[column_order]
        df_issues_all = pd.concat([df_issues_all, subset_csdb])

    if 'Demozoo' in source:
        subset_demozoo = df_demozoo[df_demozoo['title']==title].copy()
        subset_demozoo['language'] = language
        subset_demozoo['origin'] = origin
        subset_demozoo['title_cleaned'] = subset_demozoo['issue'].apply(lambda x: f'{x if not "#" in x or x.startswith("#") else x.replace("#", "No.")}')
        subset_demozoo['source'] = 'Demozoo'
        subset_demozoo['group'] = subset_demozoo['group'].apply(lambda x: '; '.join(x.split(', ')) if isinstance(x, str) else x)
        subset_demozoo['platform'] = subset_demozoo['platform'].apply(lambda x: '; '.join(x.split(', ')) if isinstance(x, str) else x)
        subset_demozoo = subset_demozoo.rename(columns={'issue': 'title_orig',
                                                        'title': 'magazine',
                                                        'release_converted': 'release_date',
                                                        'platform': 'system_detailed'})
        subset_demozoo = subset_demozoo[column_order]
        df_issues_all = pd.concat([df_issues_all, subset_demozoo])

    if 'Pouet' in source:
        subset_pouet = df_pouet[df_pouet['title']==title].copy()
        subset_pouet['language'] = language
        subset_pouet['origin'] = origin
        subset_pouet['title_cleaned'] = subset_pouet['issue'].apply(lambda x: f'{x if not "#" in x or x.startswith("#") else x.replace("#", "No.")}')
        subset_pouet['source'] = 'Pouet'
        subset_pouet['group'] = subset_pouet['group'].apply(lambda x: '; '.join(x.split(', ')) if isinstance(x, str) else x)
        subset_pouet['platform'] = subset_pouet['platform'].apply(lambda x: '; '.join(x.split(', ')) if isinstance(x, str) else x)
        subset_pouet = subset_pouet.rename(columns={'issue': 'title_orig',
                                                    'title': 'magazine',
                                                    'release_converted': 'release_date',
                                                    'platform': 'system_detailed'})
        subset_pouet = subset_pouet[column_order]
        df_issues_all = pd.concat([df_issues_all, subset_pouet])

    if 'Internet Archive' in source:
        subset_internetarchive = df_internet_archive[df_internet_archive['title']==title].copy()
        subset_internetarchive['language'] = language
        subset_internetarchive['origin'] = origin
        subset_internetarchive['title_cleaned'] = subset_internetarchive['issue'].apply(lambda x: f'{x if not "#" in x or x.startswith("#") else x.replace("#", "No.")}')
        subset_internetarchive['source'] = 'Internet Archive'
        subset_internetarchive = subset_internetarchive.rename(columns={'issue':'title_orig',
                                                                        'title': 'magazine',
                                                                        'release': 'release_date',
                                                                        'platform': 'system_detailed',
                                                                        'creator': 'group'})
        subset_internetarchive = subset_internetarchive[column_order]
        df_issues_all = pd.concat([df_issues_all, subset_internetarchive])
        
    if 'ZXpress' in source:
        subset_zxpress = df_zxpress[df_zxpress['title'] == title]
        issues = subset_zxpress['issues'].iloc[0]
        releases_zx = subset_zxpress['releases_converted'].iloc[0]
        if isinstance(issues, str):
            issues = issues.split(', ')
        else:
            issues = ['']
        if isinstance(releases_zx, str):
            releases_zx = releases_zx.split(', ')
            releases_zx = releases_zx + ['']*(len(issues)-len(releases_zx))
        else:
            releases_zx = [''] * len(issues)
        languages = [language] * len(issues)
        systems = ['ZX Spectrum'] * len(issues)
        zx_sources = ['ZXpress'] * len(issues)
        origins = [origin] * len(issues)
        links = [subset_zxpress['link'].iloc[0]] * len(issues)
        groups = [np.nan] * len(issues)
        if None in issues:
            issues_origins = [f'{title} (ZXpress)']
        else:
            issues_origins = [f'{x[1:] if x.startswith("#") else x.replace("#", "No.")}' for x in issues]
        magazine = [title] * len(issues)
        dict = {'title_cleaned': issues_origins,
                'title_orig': issues,
                'magazine': magazine,
                'release_date': releases_zx,
                'source': zx_sources,
                'language': languages,
                'origin': origins,
                'system_detailed': systems,
                'group': groups,
                'link': links}
        subset_zxpress_n = pd.DataFrame(dict)
        df_issues_all = pd.concat([df_issues_all, subset_zxpress_n])
 
df_issues_all = df_issues_all.sort_values(by='title_cleaned', key=lambda x: x.str.lower())
df_issues_all.reset_index(drop=True, inplace=True)

In [9]:
df_issues_all_copy = df_issues_all.copy()

In [10]:
df_issues_all_copy['system'] = df_issues_all_copy['system_detailed'].apply(lambda x: '; '.join(sorted(set([elem.replace(elem, systems_mapping[elem]) for elem in x.lower().split('; ')]))) if isinstance(x, str) else x)

In [11]:
df_issues_all_copy['release_date'] = pd.to_datetime(df_issues_all_copy['release_date'], format='%d.%m.%Y', errors='coerce').dt.strftime('%Y-%m-%d')

In [12]:
df_issues_all_copy = df_issues_all_copy[['title_cleaned', 'title_orig', 'magazine', 'origin', 'language','release_date', 'system', 'system_detailed', 'group', 'source', 'link']]
df_issues_all_copy['checked'] = '0'

In [13]:
# Original file
df_issues_all_copy.to_csv(f'{path_target}/east_middle_europe_magazines_issues.csv', index=False, encoding='utf-16')
# For publication and wiki
df_issues_all_copy.to_csv(f'{path_target}/east_middle_europe_magazines_issues_master.csv', index=False, encoding='utf-16')